---
title: Interactive Visualization Notebook of Key Results

---

# Interactive Visualization Notebook of Key Results

This notebook creates interactive visualizations for exploring groundwater recharge patterns, river networks, and subsurface characteristics in California's Central Valley.

## Setup and Imports

**Key Libraries:**

- **`xarray` & `rioxarray`**: Handle multi-dimensional raster data with geospatial awareness
- **`geopandas`**: Manage vector geospatial data (rivers, basins, points)
- **`ipyleaflet`**: Create interactive Leaflet.js maps in Jupyter
- **`ipywidgets`**: Build interactive UI controls (dropdowns, sliders)
- **`numpy`**: Numerical operations
- **`pandas`**: Tabular data manipulation
- **`shapely`**: Geometric operations

In [1]:
# -------------------------------
# Imports
# -------------------------------
import xarray as xr
import numpy as np
import geopandas as gpd
from ipyleaflet import (
    Map,
    ImageOverlay,
    WidgetControl,
    LayersControl,
    basemaps,
    GeoJSON,
    TileLayer,
    Heatmap,
)
import ipywidgets as widgets
from utils import leaflet_bounds, scalar_to_base64_image, find_intersections
import pandas as pd
from shapely.geometry import Point

In [2]:
# -------------------------------
# Data Paths
# -------------------------------
download_path = "./data_download"
intermediate_path = "./data_intermediate"

# Vector data (geospatial boundaries and features)
vector_rivers = f"{download_path}/data/shp/cv_rivers.geojson"
vector_subbasin = f"{download_path}/data/shp/subbasins_cv_clip.geojson"

# Point measurements
points_resistivity = f"{download_path}/data/aem/em_resistivity.csv"

# Raster/gridded data
metric_zarr_path = f"{intermediate_path}/consolidated_metric_output.zarr"

# -------------------------------
# Map Configuration
# -------------------------------
target_epsg = 4326  # WGS84 lat/lon for web mapping
center = [37.66335291403956, -120.69523554193438]  # Central Valley center
zoom = 7  # Regional zoom level
basemap = None  # Use custom tile layers instead of default basemap
map_width = "500px"
map_height = "800px"

## Configuration

Define file paths, coordinate system, and visualization parameters.

**Directory Structure:**
- `data_download/`: Raw data files (shapefiles, CSV, rasters)
- `data_intermediate/`: Processed/consolidated datasets (Zarr)

**Coordinate Reference System:**
- **EPSG:4326** (WGS84 lat/lon) - Standard for web mapping
- All data will be reprojected to this CRS for consistency

**Map Settings:**
- Center coordinates: Central Valley, California
- Zoom level: 7 (regional view)
- Map dimensions: 500×800px

In [3]:
# -------------------------------
# Load Vector Layers
# -------------------------------
rivers = gpd.read_file(vector_rivers)
rivers = rivers.to_crs(epsg=target_epsg)

subbasins = gpd.read_file(vector_subbasin)
subbasins = subbasins.to_crs(epsg=target_epsg)


def combine_rivers_gdf(river_gdf, name_column="GNIS_Name"):
    """
    Combine river fragments with the same name into single features.

    Dissolves LineString geometries that share the same river name,
    creating continuous features for visualization and analysis.

    Parameters
    ----------
    river_gdf : GeoDataFrame
        Input rivers with LineString geometries.
    name_column : str, default='GNIS_Name'
        Column containing river names for grouping.

    Returns
    -------
    GeoDataFrame
        Rivers dissolved by name with only name and geometry columns.

    Notes
    -----
    Some rivers may have gaps or duplicate names requiring QC.
    """
    rivers_combined = river_gdf.dissolve(by=name_column).reset_index()
    return rivers_combined[[name_column, "geometry"]]


# Combine river segments that share the same name
rivers = combine_rivers_gdf(rivers)
# TODO: QC this - some rivers have gaps and potential duplicate names

# Find points where rivers exit the basin boundary
river_intersections = find_intersections(rivers, subbasins)

# -------------------------------
# Load Point Data (Resistivity Profiles)
# -------------------------------
df = pd.read_csv(points_resistivity)

# Create GeoDataFrame from UTM coordinates
resistivity_profiles = gpd.GeoDataFrame(
    df,
    geometry=[Point(x, y) for x, y in zip(df["UTMX"], df["UTMY"])],
    crs="EPSG:3310",  # California Albers Equal Area
)
# TODO: Confirm EPSG code with data source documentation

# Reproject to WGS84 for web mapping
resistivity_profiles = resistivity_profiles.to_crs(f"EPSG:{target_epsg}")

# Subsample points for rendering performance
# 10,000 points provides good spatial coverage without overwhelming the map
resistivity_profiles = resistivity_profiles.sample(10000)
# TODO: Consider better approaches (clustering, server-side tiling, dynamic loading)

# -------------------------------
# Load Raster Data (Consolidated Metrics)
# -------------------------------
ds = xr.open_zarr(metric_zarr_path)

# Ensure consistent dimension ordering for visualization
ds = ds.transpose("fraction", "y", "x")
ds = ds.sortby("y", ascending=False)  # Top to bottom (north to south)
ds = ds.sortby("x", ascending=True)  # Left to right (west to east)

# Reproject raster from CA Albers to WGS84
ds.rio.write_crs(3310, inplace=True)  # Set source CRS
ds_reprojected = ds.rio.reproject(f"EPSG:{target_epsg}")

# -------------------------------
# Optimize Geometries for Map Performance
# -------------------------------
# Simplify geometries with ~100m tolerance (≈0.001° at this latitude)
# Reduces vertex count while preserving visual appearance and topology
subbasins["geometry"] = subbasins.geometry.simplify(tolerance=0.001, preserve_topology=True)
rivers["geometry"] = rivers.geometry.simplify(tolerance=0.001, preserve_topology=True)

## Data Loading and Preprocessing

Load all datasets and prepare them for visualization by:
1. Reprojecting to common CRS (EPSG:4326)
2. Processing geometries for performance
3. Computing derived features

### Processing Steps

**Vector Data (Rivers & Basins):**
- Load GeoJSON files with `geopandas`
- Dissolve river fragments with same name into single geometries
- Simplify geometries (~100m tolerance) for faster rendering
- Find river-basin boundary intersections (exit/inflow points)

**Point Data (Resistivity):**
- Load CSV with UTM coordinates
- Convert to GeoDataFrame with proper CRS (EPSG:3310 → 4326)
- Subsample to 10,000 points for performance

**Raster Data (Metrics):**
- Open Zarr dataset with `xarray`
- Reorder and sort dimensions for proper display
- Reproject from CA Albers (EPSG:3310) to WGS84 (EPSG:4326)

**Performance Optimizations:**
- Geometry simplification reduces vertex count
- Point subsampling reduces render time
- Lazy loading with Zarr for large datasets

In [ ]:
# -------------------------------
# Basemap Tile Layers
# -------------------------------
# Ocean basemap with bathymetry from ESRI ArcGIS Online
l_ocean = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}",
    name="Ocean/Water",
    opacity=1.0,
)

# Hillshade elevation for terrain context
l_elevation = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}",
    name="Hillshade",
    opacity=1.0,
)

# -------------------------------
# Vector Layers (Lines & Polygons)
# -------------------------------
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={"color": "blue", "weight": 1, "opacity": 0.7},
    name="Rivers",
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={"color": "black", "weight": 1, "fill": False},
    name="Subbasins",
)

# River exit/inflow points at basin boundary
intersections_layer = GeoJSON(
    data=river_intersections.__geo_interface__,
    point_style={
        "radius": 6,
        "color": "orange",
        "fillColor": "orange",
        "fillOpacity": 0.5,
        "width": 2
    },
    name="River Exits",
)

## Layer Creation

Create all map layers for interactive visualization. Each layer can be toggled on/off independently.

### Layer Types

**Tile Layers (Basemaps):**
- `ipyleaflet.TileLayer` fetches map tiles from remote servers
- Ocean layer: Shows water features and bathymetry
- Hillshade layer: Provides terrain context with shaded relief

**Vector Layers:**
- `ipyleaflet.GeoJSON` renders vector features client-side
- Rivers: Blue lines showing drainage network
- Subbasins: Black outlines defining hydrologic boundaries
- Intersections: Orange markers at river exit points

**Point Visualization:**
- **GeoJSON points**: Individual markers (very small radius for dense data)
- **Heatmap**: Density visualization using kernel density estimation
  - `radius`: Influence distance of each point (pixels)
  - `blur`: Smoothing factor
  - `gradient`: Color scale from low to high density

**Use Cases:**
- Points: Precise locations
- Heatmaps: Spatial patterns and sampling density

In [8]:
#| label: fig:multi-maps

# -------------------------------
# Dual Map Controller Class
# -------------------------------

from ipyleaflet import Map, GeoJSON, WidgetControl, LayersControl, basemaps, Popup
import ipywidgets as widgets
import geopandas as gpd
from IPython.display import display
from ipywidgets import jslink
from utils import create_colorbar_widget

param_dict = {
    'fraction_coarse': {
        'vmin':0, 'vmax':100, 'short_label': 'FCD' , 'label': 'Fraction of coarse-dominated sediment (%)', 'cmap':'RdBu_r', 'log_scale':False
    },
    'path_length_norm': {
        'vmin':0, 'vmax':100,  'short_label': 'Normalized distance', 'label': 'Normalized path length', 'cmap':'Greens', 'log_scale':True
    },
    'path_to_no_flow': {
        'vmin':0, 'vmax':20, 'short_label': 'Depth to no flow','label': 'Depth to shallowest no-flow unit\n or base surface (m)', 'cmap':'Greens', 'log_scale':False
    },
}


class DualMapController:
    """
    Controller for synchronized dual maps with interactive controls.

    Manages two side-by-side ipyleaflet maps with:
    - Synchronized zoom and pan
    - Independent data layers
    - Interactive feature highlighting
    - Dynamic dataset selection
    - Dynamic colorbars
    """

    def __init__(
        self,
        width,
        height,
        center,
        zoom,
        subbasins,
        subbasin_column,
        rivers_gdf,
        river_intersections_gdf,
        param_dict,
    ):
        """Initialize dual map controller with configuration and data."""
        self.subbasins = subbasins
        self.subbasin_column = subbasin_column
        self.rivers_gdf = rivers_gdf
        self.river_intersections_gdf = river_intersections_gdf
        self.param_dict = param_dict

        # Create maps with explicit center and zoom
        layout = widgets.Layout(width=width, height=height)
        self.m1 = Map(
            center=center, zoom=zoom, basemap=basemaps.CartoDB.Positron, layout=layout
        )
        self.m2 = Map(
            center=center, zoom=zoom, basemap=basemaps.CartoDB.Positron, layout=layout
        )

        # Synchronize views using JavaScript linking
        jslink((self.m1, "center"), (self.m2, "center"))
        jslink((self.m1, "zoom"), (self.m2, "zoom"))

        # -------------------------------
        # Define Highlight Styles
        # -------------------------------
        # Subbasin highlight style (orange dashed border)
        self.highlight_style = {
            "color": "orange",
            "weight": 3,
            "opacity": 0.8,
            "fillColor": "orange",
            "fillOpacity": 0.1,
            "dashArray": "5, 5",
        }

        # River highlight style (bright blue glow effect)
        self.river_highlight_style = {
            "color": "#0099ff",
            "weight": 5,
            "opacity": 0.8,
            "lineCap": "round",
            "lineJoin": "round",
        }

        # -------------------------------
        # State Tracking
        # -------------------------------
        # Track highlight layers (created on demand)
        self.river_highlight_m1 = None
        self.river_highlight_m2 = None
        self.current_highlight = None  # Subbasin highlight

        # Track popup references for cleanup
        self.current_popup_m1 = None
        self.current_popup_m2 = None

        # Widget for displaying selected river name
        self.river_name_widget = widgets.HTML(value="")

        # Debug output widget
        self.debug_output = widgets.Output()
        
        # Colorbar widgets and controls (initialized later)
        self.colorbar_control_m1 = None
        self.colorbar_control_m2 = None

    def add_base_layers(self, rivers, subbasins, fcd_layer, scalar_overlay):
        """Add base layers to both maps."""
        # Add to both maps
        self.m1.add_layer(rivers)
        self.m2.add_layer(rivers)
        self.m1.add_layer(subbasins)
        self.m2.add_layer(subbasins)

        # Map-specific overlays
        self.m1.add_layer(fcd_layer)
        self.m2.add_layer(scalar_overlay)

        # Store references for dynamic updates
        self.fcd_layer = fcd_layer
        self.scalar_overlay = scalar_overlay

    def _create_colorbar_m1(self, dataset_name):
        """Create/update colorbar for left map (FCD)."""
        params = self.param_dict[dataset_name]
        
        # Remove existing colorbar if present
        if self.colorbar_control_m1 is not None:
            self.m1.remove_control(self.colorbar_control_m1)
        
        # Create new colorbar
        colorbar_widget = create_colorbar_widget(
            vmin=params['vmin'],
            vmax=params['vmax'],
            cmap=params['cmap'],
            label=params['label'],
            width=80,
            height=300,
            bar_width=0.1,
        )
        
        # Add colorbar at topright (on top of dropdown)
        self.colorbar_control_m1 = WidgetControl(
            widget=colorbar_widget, 
            position='topright'
        )
        self.m1.add_control(self.colorbar_control_m1)
    
    def _create_colorbar_m2(self, dataset_name):
        """Create/update colorbar for right map."""
        params = self.param_dict[dataset_name]
        
        # Remove existing colorbar if present
        if self.colorbar_control_m2 is not None:
            self.m2.remove_control(self.colorbar_control_m2)
        
        # Create new colorbar
        colorbar_widget = create_colorbar_widget(
            vmin=params['vmin'],
            vmax=params['vmax'],
            cmap=params['cmap'],
            label=params['label'],
            width=80,
            height=300,
            bar_width=0.1,
        )
        
        # Add colorbar at topright (on top of dropdown)
        self.colorbar_control_m2 = WidgetControl(
            widget=colorbar_widget, 
            position='topright'
        )
        self.m2.add_control(self.colorbar_control_m2)
    
    def _update_layer_name(self, layer, dataset_name):
        """Update layer name using short_label from param_dict."""
        if dataset_name in self.param_dict:
            layer.name = self.param_dict[dataset_name]['short_label']

    def add_river_intersections(self, intersections_layer):
        """Add clickable river intersection markers to both maps."""
        self.intersections_layer = intersections_layer

        # Register click event handler
        self.intersections_layer.on_click(self._on_intersection_click)

        # Add to both maps
        self.m1.add_layer(self.intersections_layer)
        self.m2.add_layer(self.intersections_layer)

    def _on_intersection_click(self, feature, **kwargs):
        """
        Handle click on river intersection point.

        Highlights the full river geometry on both maps and displays
        a popup with the river name.
        """
        river_name = feature["properties"].get("river_name", "Unknown")
        coords = feature["geometry"]["coordinates"]

        # Update river name display
        self.river_name_widget.value = f"<b>Selected River:</b> {river_name}"

        # -------------------------------
        # Clean Up Previous Popups
        # -------------------------------
        if self.current_popup_m1 is not None:
            self.m1.remove_layer(self.current_popup_m1)
        if self.current_popup_m2 is not None:
            self.m2.remove_layer(self.current_popup_m2)

        # -------------------------------
        # Create New Popups
        # -------------------------------
        popup_html = widgets.HTML(f"<b>{river_name}</b>")
        self.current_popup_m1 = Popup(
            location=(coords[1], coords[0]),
            child=popup_html,
            close_button=True,
            auto_close=False,
            close_on_escape_key=True,
            name="River Info Popup",
        )

        popup_html2 = widgets.HTML(f"<b>{river_name}</b>")
        self.current_popup_m2 = Popup(
            location=(coords[1], coords[0]),
            child=popup_html2,
            close_button=True,
            auto_close=False,
            close_on_escape_key=True,
            name="River Info Popup",
        )

        self.m1.add_layer(self.current_popup_m1)
        self.m2.add_layer(self.current_popup_m2)

        # -------------------------------
        # Highlight River Geometry
        # -------------------------------
        matching_river = self.rivers_gdf[self.rivers_gdf["GNIS_Name"] == river_name]

        if not matching_river.empty:
            # Remove previous river highlights
            if self.river_highlight_m1 is not None:
                self.m1.remove_layer(self.river_highlight_m1)
            if self.river_highlight_m2 is not None:
                self.m2.remove_layer(self.river_highlight_m2)

            # Create new highlight layers
            self.river_highlight_m1 = GeoJSON(
                data=matching_river.__geo_interface__,
                style=self.river_highlight_style,
                name="River Highlight",
            )
            self.river_highlight_m2 = GeoJSON(
                data=matching_river.__geo_interface__,
                style=self.river_highlight_style,
                name="River Highlight",
            )

            self.m1.add_layer(self.river_highlight_m1)
            self.m2.add_layer(self.river_highlight_m2)

    def add_controls(self):
        """Add LayersControl to both maps (call AFTER adding base layers)."""
        self.m1.add_control(LayersControl(position="bottomleft", collapsed=False))
        self.m2.add_control(LayersControl(position="bottomleft", collapsed=False))

    def create_dataset_selector(self, ds, initial_dataset):
        """Create dataset selection controls for right map."""
        self.ds = ds
        self.current_dataset_name = initial_dataset
        self.da_scalar = ds[initial_dataset]

        # -------------------------------
        # Dataset Dropdown (use short_label)
        # -------------------------------
        # Create options mapping from short_label to dataset name
        dataset_options = {
            self.param_dict[v]['short_label']: v 
            for v in ds.data_vars if v not in ["fraction_coarse"]
        }
        
        dataset_dropdown = widgets.Dropdown(
            options=list(dataset_options.keys()),
            value=self.param_dict[initial_dataset]['short_label'],
            description="Dataset:",
            style={"description_width": "initial"},
        )
        
        # Store mapping for callback
        self.dataset_options = dataset_options
        dataset_dropdown.observe(self._on_dataset_change, names="value")

        # -------------------------------
        # Threshold Slider
        # -------------------------------
        self.slider = widgets.SelectionSlider(
            options=ds.fraction.values,
            value=20,
            description="FCD Threshold (%)",
            style={"description_width": "initial"},
        )
        self.slider.observe(self._on_threshold_change, names="value")

        # Add dropdown FIRST (will be at back)
        controls = widgets.VBox([dataset_dropdown, self.slider])
        widget_control = WidgetControl(widget=controls, position="topright")
        self.m2.add_control(widget_control)
        
        # Add colorbar AFTER (will be on top)
        self._create_colorbar_m2(initial_dataset)

    def _on_dataset_change(self, change):
        """Handle dataset selection change."""
        # Map from short_label back to dataset name
        short_label = change["new"]
        self.current_dataset_name = self.dataset_options[short_label]
        self.da_scalar = self.ds[self.current_dataset_name]
        
        # Update layer name
        self._update_layer_name(self.scalar_overlay, self.current_dataset_name)
        
        # Update colorbar
        self._create_colorbar_m2(self.current_dataset_name)
        
        # Update overlay
        self._update_scalar_overlay()

    def _on_threshold_change(self, change):
        """Handle threshold slider change."""
        self._update_scalar_overlay()

    def _update_scalar_overlay(self):
        """Update right map overlay with new dataset/threshold."""
        threshold = self.slider.value
        params = self.param_dict[self.current_dataset_name]

        # Select data by threshold if dimension exists
        if "fraction" in self.da_scalar.dims:
            da_overlay = self.da_scalar.sel(fraction=threshold)
        else:
            da_overlay = self.da_scalar

        # Update image overlay with new data using param_dict values
        self.scalar_overlay.url = scalar_to_base64_image(
            da_overlay,
            cmap=params['cmap'],
            vmin=params['vmin'],
            vmax=params['vmax'],
        )

    def create_subbasin_selector(self, center, zoom):
        """Create subbasin selection dropdown for left map."""
        self.default_center = center
        self.default_zoom = zoom

        # Create dropdown options
        subbasin_names = ["All Regions"] + self.subbasins[
            self.subbasin_column
        ].tolist()
        dropdown = widgets.Dropdown(
            options=subbasin_names,
            description="Subbasin:",
            style={"description_width": "initial"},
        )
        dropdown.observe(self._on_subbasin_change, names="value")

        # Add dropdown FIRST (will be at back)
        controls_vbox = widgets.VBox([dropdown, self.river_name_widget])
        control = WidgetControl(widget=controls_vbox, position="topright")
        self.m1.add_control(control)
        
        # Add colorbar AFTER (will be on top)
        self._create_colorbar_m1('fraction_coarse')

    def _on_subbasin_change(self, change):
        """Handle subbasin selection change - zooms and highlights region."""
        selected_name = change["new"]

        # Remove previous highlight
        if self.current_highlight is not None:
            self.m1.remove_layer(self.current_highlight)
            self.m2.remove_layer(self.current_highlight)
            self.current_highlight = None

        if selected_name == "All Regions":
            # Reset to default view
            self.m1.center = self.default_center
            self.m1.zoom = self.default_zoom
        else:
            # Find selected subbasin
            selected_subbasin = self.subbasins[
                self.subbasins[self.subbasin_column] == selected_name
            ]

            if selected_subbasin.empty:
                with self.debug_output:
                    print("No matching subbasin found!")
                return

            # Zoom to subbasin bounds
            bounds = selected_subbasin.total_bounds
            self.m1.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

            # Create highlight overlay
            self.current_highlight = GeoJSON(
                data=selected_subbasin.__geo_interface__,
                style=self.highlight_style,
                name="Region Highlight",
            )
            self.m1.add_layer(self.current_highlight)
            self.m2.add_layer(self.current_highlight)

    def display(self):
        """Display the dual map setup."""
        display(self.debug_output)
        display(widgets.HBox([self.m1, self.m2]))


# -------------------------------
# Dual Map Visualization
# -------------------------------

# Initialize controller
controller = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
)

# -------------------------------
# Create Raster Overlays
# -------------------------------
# Left map: Fraction Coarse Dominated (static reference)
fcd_params = param_dict['fraction_coarse']
fcd_ave = ds_reprojected["fraction_coarse"]
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(
        fcd_ave, 
        cmap=fcd_params['cmap'], 
        vmin=fcd_params['vmin'], 
        vmax=fcd_params['vmax']
    ),
    bounds=leaflet_bounds(fcd_ave),
    opacity=1.0,
    name=fcd_params['short_label'],  # Use short_label from param_dict
)

# Right map: Dynamic metric overlay (switchable)
current_dataset_name = "path_length_norm"
current_params = param_dict[current_dataset_name]
da_scalar = ds_reprojected[current_dataset_name].sel(fraction=20).load()
scalar_overlay = ImageOverlay(
    url=scalar_to_base64_image(
        da_scalar, 
        cmap=current_params['cmap'],
        vmin=current_params['vmin'],
        vmax=current_params['vmax']
    ),
    bounds=leaflet_bounds(da_scalar),
    opacity=1.0,
    name=current_params['short_label'],  # Use short_label from param_dict
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
controller.add_base_layers(l_rivers, l_subbasins, fcd_layer, scalar_overlay)
controller.add_controls()  # Must be called AFTER adding base layers
controller.add_river_intersections(intersections_layer)
controller.create_dataset_selector(ds_reprojected, current_dataset_name)
controller.create_subbasin_selector(center=center, zoom=zoom)
controller.display()

Output()

## Figures 2-7: Dual Synchronized Maps

Side-by-side interactive maps with synchronized views for comparative analysis.

### Architecture

**DualMapController Class:**
A custom controller that manages two linked maps with shared navigation and independent data layers.

**Key Features:**
1. **View Synchronization**: Zoom and pan are linked via `ipywidgets.jslink()`
2. **Independent Layers**: Each map displays different data for comparison
3. **Interactive Highlighting**: Click features to highlight across both maps
4. **Dynamic Data Updates**: Switch datasets and thresholds in real-time

### Map Layout

**Left Map (m1):**
- Fraction Coarse Dominated (FCD) raster overlay
- Subbasin selector with zoom-to-region
- River exit point markers (clickable)
- Shows static reference data

**Right Map (m2):**
- Switchable metric datasets (path length, recharge, etc.)
- Threshold slider for FCD filtering
- Same base layers and markers
- Shows dynamic analysis results

### Interactive Workflows

**Subbasin Selection:**
1. Choose from dropdown → map zooms to bounds
2. Subbasin outline highlighted with orange dashed border
3. Highlight persists until new selection
4. "All Regions" returns to default view

**River Selection:**
1. Click orange marker at river exit point
2. Full river geometry highlights in bright blue on both maps
3. Popup shows river name
4. Multiple river highlights overlay (compare drainage patterns)

**Dataset Exploration:**
1. Select metric from dropdown (right map only)
2. Adjust FCD threshold with slider
3. Right map updates raster overlay
4. Left map unchanged for reference comparison

### Technical Implementation

**Synchronization:**
```python
jslink((map1, 'center'), (map2, 'center'))
jslink((map1, 'zoom'), (map2, 'zoom'))
```
- JavaScript-level linking for smooth interaction
- No Python callback overhead
- Bi-directional synchronization

**Event Handlers:**
- `on_click` for feature selection
- `.observe()` for widget changes
- Layer management (add/remove) for highlights

**State Management:**
- Track current highlights to prevent duplicates
- Store popup references for cleanup
- Maintain active dataset/threshold selections

In [7]:
import numpy as np
import xarray as xr
from ipyleaflet import Map, ImageOverlay, WidgetControl
import ipywidgets as widgets
from matplotlib import cm
from matplotlib.colors import Normalize
from PIL import Image
from io import BytesIO
import base64

# ========================================
# 1. CREATE SYNTHETIC RASTER DATA
# ========================================
# Create sample data (replace with your actual data)
lats = np.linspace(37.0, 38.0, 100)
lons = np.linspace(-121.0, -120.0, 100)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Generate synthetic temperature data
data = 20 + 10 * np.sin(lon_grid * 5) * np.cos(lat_grid * 5)

# Create xarray DataArray
da = xr.DataArray(
    data,
    coords={'lat': lats, 'lon': lons},
    dims=['lat', 'lon'],
    name='temperature'
)

# ========================================
# 2. HELPER FUNCTIONS
# ========================================
def scalar_to_base64_image(array, cmap='viridis', vmin=None, vmax=None):
    """Convert 2D array to base64-encoded PNG with colormap."""
    if vmin is None:
        vmin = np.nanmin(array)
    if vmax is None:
        vmax = np.nanmax(array)
    
    # Apply colormap
    norm = Normalize(vmin=vmin, vmax=vmax)
    mapper = cm.ScalarMappable(norm=norm, cmap=cmap)
    rgba = mapper.to_rgba(array, bytes=False)
    
    # Make NaN transparent
    mask = np.isnan(array)
    rgba[mask, 3] = 0
    
    # Convert to image
    rgba_uint8 = (rgba * 255).astype(np.uint8)
    img = Image.fromarray(rgba_uint8, mode='RGBA')
    
    # Encode as base64
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_str = "data:image/png;base64," + base64.b64encode(buffer.getvalue()).decode()
    return img_str

from utils import create_colorbar_widget

# ========================================
# 3. SETUP MAP AND DATA
# ========================================
# Calculate bounds for ipyleaflet
bounds = [
    [da.lat.min().item(), da.lon.min().item()],
    [da.lat.max().item(), da.lon.max().item()]
]

# Data range for colorbar
vmin = float(da.min())
vmax = float(da.max())
cmap_name = 'RdYlBu_r'

# Create map
m = Map(
    center=[37.5, -120.5],
    zoom=9,
    scroll_wheel_zoom=True,
    layout=widgets.Layout(width='700px', height='500px')
)

# Create raster overlay
overlay = ImageOverlay(
    url=scalar_to_base64_image(da.values, cmap=cmap_name, vmin=vmin, vmax=vmax),
    bounds=bounds,
    opacity=0.7,
    name='Temperature',

)
m.add_layer(overlay)

# ========================================
# 4. CREATE COLORBAR
# ========================================
colorbar = create_colorbar_widget(
    vmin=vmin,
    vmax=vmax,
    cmap=cmap_name,
    label='Temperature (°C)',
    bar_width=0.1,
)

# Add colorbar to map
colorbar_control = WidgetControl(widget=colorbar, position='bottomright')
m.add_control(colorbar_control)


# ========================================
# 6. DISPLAY MAP
# ========================================
display(m)

/var/folders/3b/rlk8rxgs1xx44kmtzfrwlvth0000gn/T/ipykernel_37183/3080798157.py:51: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(rgba_uint8, mode='RGBA')


Map(center=[37.5, -120.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…